In [6]:
import openpyxl
from openpyxl import load_workbook
import os

# File paths
zra_file = "/Users/malvernbright/Documents/Young Phiroz/product_import_template (3).xlsx"
product_file = "/Users/malvernbright/Documents/Young Phiroz/Product (product.template).xlsx"

print("Loading workbooks...")

# Load both workbooks
zra_wb = load_workbook(zra_file)
product_wb = load_workbook(product_file)

# Get active sheets
zra_sheet = zra_wb.active
product_sheet = product_wb.active

# Get headers from both sheets
zra_headers = {}
product_headers = {}

# Map ZRA headers (row 1)
for col in range(1, zra_sheet.max_column + 1):
    header = zra_sheet.cell(row=1, column=col).value
    if header:
        zra_headers[header] = col

# Map Product headers (row 1)
for col in range(1, product_sheet.max_column + 1):
    header = product_sheet.cell(row=1, column=col).value
    if header:
        product_headers[header] = col

# Check if barcode column exists, if not add it
if 'barcode' not in product_headers:
    print("Adding 'barcode' column to Product template...")
    next_col = product_sheet.max_column + 1
    product_sheet.cell(row=1, column=next_col).value = 'barcode'
    product_headers['barcode'] = next_col

print(f"ZRA file has {len(zra_headers)} columns")
print(f"Product file has {len(product_headers)} columns")

# Create mapping between ZRA columns and Product columns
column_mapping = {
    # ZRA Column Name: Product Column Name
    'Barcode': 'barcode',  # Added barcode mapping
    'Item Classification Code(zra)': 'x_itemClsCd',
    'Origin Place Code (Nation)': 'x_orgnCd',
    'Origin Place (Nation)': 'x_orgnNat',
    'Origin Place Code (Nation)': 'x_orgnNatCd',
    'Packaging Unit Code (zra)': 'x_PacCd',
    'Packaging Unit': 'x_pkgUnit',
    'VAT Category Code': 'x_vatCatCd',
    'ZRA Product Type': 'x_tlCatCd',
    'Quantity Unit Code(zra)': 'x_qtyUnitCd',
    'Insurance Applicable (Y/N)': 'x_isrcAplcbYn',
    'Indication of whether an item has rental or not': 'x_rentalYn',
    'Indication of whether an item has service charge': 'x_svcChargeYn',
    'ZRA Modify Y/N': 'x_ZRAModYn',
    'Item Classification': 'x_itemCls',
    'Export Nation Code': 'x_exptNatCd',
    'Export Nation': 'x_exptNat',
}

# Count products to process
total_products = 0
for row in range(2, product_sheet.max_row + 1):
    product_name = product_sheet.cell(row=row, column=product_headers.get('name', 10)).value
    if product_name:
        total_products += 1

print(f"\nFound {total_products} products in Product template")
print(f"Found {zra_sheet.max_row - 1} products in ZRA template\n")

# Create a dictionary to quickly lookup ZRA data by product name
zra_data_dict = {}
name_col = zra_headers.get('Product name', 1)

for row in range(2, zra_sheet.max_row + 1):
    product_name = zra_sheet.cell(row=row, column=name_col).value
    if product_name:
        # Store the row number for this product
        zra_data_dict[product_name.strip().upper()] = row

print("Processing products and filling ZRA data...")

# Track statistics
matched = 0
unmatched = 0
updated_fields = 0
duplicate_barcodes = {}
skipped_duplicates = 0

# First pass: collect all barcodes to detect duplicates
print("Checking for duplicate barcodes...")
barcode_usage = {}
barcode_col = zra_headers.get('Barcode', 3)

for row in range(2, zra_sheet.max_row + 1):
    barcode = zra_sheet.cell(row=row, column=barcode_col).value
    product_name = zra_sheet.cell(row=row, column=name_col).value
    
    if barcode and product_name:
        barcode_str = str(barcode).strip()
        if barcode_str:
            if barcode_str not in barcode_usage:
                barcode_usage[barcode_str] = []
            barcode_usage[barcode_str].append(product_name)

# Find duplicates
for barcode, products in barcode_usage.items():
    if len(products) > 1:
        duplicate_barcodes[barcode] = products
        print(f"Warning: Barcode {barcode} is used by {len(products)} products: {', '.join(products[:3])}{'...' if len(products) > 3 else ''}")

if duplicate_barcodes:
    print(f"\nFound {len(duplicate_barcodes)} duplicate barcodes. These will be skipped to avoid conflicts.")
    print("You may need to manually correct these in your ZRA data.\n")

# Process each product in the Product template
for row in range(2, product_sheet.max_row + 1):
    product_name_cell = product_headers.get('name', 10)
    product_name = product_sheet.cell(row=row, column=product_name_cell).value
    
    if not product_name:
        continue
    
    # Look up this product in ZRA data
    lookup_name = product_name.strip().upper()
    zra_row = zra_data_dict.get(lookup_name)
    
    if zra_row:
        matched += 1
        
        # Copy data from ZRA to Product template based on mapping
        for zra_col_name, product_col_name in column_mapping.items():
            if zra_col_name in zra_headers and product_col_name in product_headers:
                zra_col = zra_headers[zra_col_name]
                product_col = product_headers[product_col_name]
                
                # Get value from ZRA sheet
                value = zra_sheet.cell(row=zra_row, column=zra_col).value
                
                # Special handling for barcode - skip if duplicate
                if zra_col_name == 'Barcode' and value:
                    barcode_str = str(value).strip()
                    if barcode_str in duplicate_barcodes:
                        print(f"Skipping duplicate barcode {barcode_str} for product: {product_name}")
                        skipped_duplicates += 1
                        continue
                
                # Set value in Product sheet
                if value is not None:
                    product_sheet.cell(row=row, column=product_col).value = value
                    updated_fields += 1
        
        if matched % 100 == 0:
            print(f"Processed {matched} products...")
    else:
        unmatched += 1

print(f"\n=== Summary ===")
print(f"Total products processed: {matched + unmatched}")
print(f"Matched and updated: {matched}")
print(f"Not found in ZRA data: {unmatched}")
print(f"Total fields updated: {updated_fields}")
print(f"Duplicate barcodes skipped: {skipped_duplicates}")

if duplicate_barcodes:
    print(f"\n=== Duplicate Barcodes Found ===")
    print(f"These {len(duplicate_barcodes)} barcodes need manual correction:")
    for barcode, products in list(duplicate_barcodes.items())[:10]:
        print(f"  {barcode}: {', '.join(products[:2])}{'...' if len(products) > 2 else ''}")
    if len(duplicate_barcodes) > 10:
        print(f"  ... and {len(duplicate_barcodes) - 10} more")

# Save the updated Product template
print(f"\nSaving updated Product template...")
output_file = "/Users/malvernbright/Documents/Young Phiroz/Product (product.template) - UPDATED.xlsx"
# output_file = "/Users/malvernbright/Desktop/ML/Young Phiroz/Product (product.template) - UPDATED.xlsx"
product_wb.save(output_file)

print(f"✓ File saved successfully: {output_file}")
print("\nYou can now import this updated file into Odoo!")

# Close workbooks
zra_wb.close()
product_wb.close()

Loading workbooks...
Adding 'barcode' column to Product template...
ZRA file has 35 columns
Product file has 70 columns

Found 993 products in Product template
Found 2167 products in ZRA template

Processing products and filling ZRA data...
Checking for duplicate barcodes...
Processed 100 products...
Processed 200 products...
Processed 300 products...
Processed 400 products...
Processed 500 products...
Processed 600 products...
Processed 700 products...
Processed 800 products...
Processed 900 products...

=== Summary ===
Total products processed: 993
Matched and updated: 989
Not found in ZRA data: 4
Total fields updated: 6126
Duplicate barcodes skipped: 0

Saving updated Product template...
✓ File saved successfully: /Users/malvernbright/Documents/Young Phiroz/Product (product.template) - UPDATED.xlsx

You can now import this updated file into Odoo!


In [9]:
import numpy as np
import pandas as pd

In [10]:
df = pd.read_excel('/Users/malvernbright/Desktop/ML/product_template.xlsx', sheet_name='Sheet1')

In [11]:
df.head()

,id,activity_exception_decoration,combo_ids,standard_price,is_favorite,virtual_available,x_studio_ic,default_code,name,qty_available,...,tax_string,x_taxTyCd,x_tlCatCd,x_useYn,x_vatCatCd,x_ZRAModYn,x_itemTyCd,id.1,zra_purchase_uom_id,zra_uom_id
0,__export__.product_template_63201_3e20ff9a,NaN,NaN,0.0,False,0,NaN,NaN,NAOMI SHOWER GEL OUD & ROSE 1L,0,...,(= 1.16 ZK Incl. Taxes),NaN,2.0,Yes,A,Y,NaN,__export__.product_template_63201_3e20ff9a,NaN,NaN
1,__export__.product_template_63984_e7c25b74,NaN,NaN,0.0,False,0,NaN,NaN,AL SHAFAA NATURAL HONEY 500G,0,...,(= 52.20 ZK Incl. Taxes),NaN,2.0,Yes,A,Y,NaN,__export__.product_template_63984_e7c25b74,NaN,NaN
2,__export__.product_template_65187_df2f0e26,NaN,NaN,0.0,False,0,Unclassified product [4332255500],NaN,AL-RISHENG LORD PERFUME 100ML,0,...,(= 33.64 ZK Incl. Taxes),A,2.0,Yes,A,Y,Finished Product,__export__.product_template_65187_df2f0e26,Pieces/item [Number],Pieces/item [Number]
3,__export__.product_template_63202_6c4e0140,NaN,NaN,0.0,False,0,Dried cut santa fe rose [10402765],NaN,AL-RISHENG LORD PERFUME 100ML,0,...,(= 1.16 ZK Incl. Taxes),A,2.0,Yes,A,Y,Finished Product,__export__.product_template_63202_6c4e0140,Each,Each
4,__export__.product_template_65097_ec35d219,NaN,NaN,0.0,False,0,NaN,NaN,ALOHA PO,0,...,(= 1.16 ZK Incl. Taxes),NaN,NaN,Yes,A,Yes,NaN,__export__.product_template_65097_ec35d219,NaN,NaN


In [12]:
df['x_btchNo']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
988   NaN
989   NaN
990   NaN
991   NaN
992   NaN
Name: x_btchNo, Length: 993, dtype: float64